## Se descarga el codigo de Kaggle, ya que este viene de multiples fuentes

In [5]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'airlines-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3852902%2F6678321%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240316%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240316T204423Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D34977f754eb8e44bc57f195201298202fa1d417af300afa69bc0e2dff519c742ef5c069229a809481a5f7ab5afede699612da99721bcbe932aede9fe505439b9af9272a547dec4de1c29f6421d037c5a950c3dcd320e3618a768b83399c562aa32f9ce157824eed66bea4be91bc5e711f95b05d8319b03d486c02bba03e366b24c103773a4befdd2e219fbdf4967715347bd5dd62a579ab259a3efd7e8e221af8e1a51765fc96ac3fddee69f9db34082fb754b8079aff14d24f9bfeb72fa2ca11fed3ce0c980a394ce824bd7104fc19055a882aa3bc93d0c0ccb82cc69c83eddac7b409c3062cd0e6eec5b9df4f87aeaefeb75f8f2fbb1d303781c343b24dece'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 31061593 bytes downloaded
Downloaded and uncompressed: airlines-dataset
Data source import complete.


### Luego se importan varias bibliotecas a usar en el proyecto

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/airlines-dataset/travel.sqlite


## Luego importa las bibliotecas necesarias para el análisis de datos

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

path = "../input/airlines-dataset/"  #Insert path here
database = path + 'travel.sqlite'

## Creamos la conexión con la  base de datos

In [3]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,aircrafts_data,aircrafts_data,2,CREATE TABLE aircrafts_data (\r\n aircraft_...
1,table,airports_data,airports_data,3,CREATE TABLE airports_data (\r\n airport_co...
2,table,boarding_passes,boarding_passes,4,CREATE TABLE boarding_passes (\r\n ticket_n...
3,table,bookings,bookings,5,CREATE TABLE bookings (\r\n book_ref charac...
4,table,flights,flights,6,CREATE TABLE flights (\r\n flight_id intege...
5,table,seats,seats,7,CREATE TABLE seats (\r\n aircraft_code char...
6,table,ticket_flights,ticket_flights,8,CREATE TABLE ticket_flights (\r\n ticket_no...
7,table,tickets,tickets,9,CREATE TABLE tickets (\r\n ticket_no charac...


# ii. Para el dataset de aviones responder:

## 1. ¿En el dataset de aviones: que tipos de asientos y tiquetes hay?

In [47]:
seats = pd.read_sql("SELECT * FROM seats", conn)

print("\n")

print("Tipos de Asientos Disponibles:")
print("\n")
tipos_asiento = seats['fare_conditions'].value_counts()
print(tipos_asiento)



Tipos de Asientos Disponibles:


Economy     1139
Business     152
Comfort       48
Name: fare_conditions, dtype: int64


## 2. ¿Qué modelos de aviones hay?


In [46]:
aircrafts_data = pd.read_sql("SELECT * FROM aircrafts_data", conn)

print("Modelos de Aviones Disponibles:")
print("\n")
modelo_aviones = aircrafts_data['model'].value_counts()
print(modelo_aviones)

Modelos de Aviones Disponibles:


{"en": "Boeing 777-300", "ru": "Боинг 777-300"}               1
{"en": "Boeing 767-300", "ru": "Боинг 767-300"}               1
{"en": "Sukhoi Superjet-100", "ru": "Сухой Суперджет-100"}    1
{"en": "Airbus A320-200", "ru": "Аэробус A320-200"}           1
{"en": "Airbus A321-200", "ru": "Аэробус A321-200"}           1
{"en": "Airbus A319-100", "ru": "Аэробус A319-100"}           1
{"en": "Boeing 737-300", "ru": "Боинг 737-300"}               1
{"en": "Cessna 208 Caravan", "ru": "Сессна 208 Караван"}      1
{"en": "Bombardier CRJ-200", "ru": "Бомбардье CRJ-200"}       1
Name: model, dtype: int64


##3. ¿Cuántas reservas hay?

In [61]:
bookings = pd.read_sql("SELECT * FROM bookings", conn)

print("Cantidad Total de Reservas:")
print("\n")
cantidad_reservas = len(bookings)
print(cantidad_reservas)

Cantidad Total de Reservas:


262788


##4. ¿Cuál es el precio medio de las reservas? ¿el más caro? ¿El más barato?


In [64]:
bookings = pd.read_sql("SELECT * FROM bookings", conn)

print("El precio medio de las reservas es:")
print("\n")
reservas = bookings['total_amount'].mean()
print("$", reservas)
print("\n")
print("El precio mas alto de las reservas es:")
print("\n")
reservas = bookings['total_amount'].max()
print("$", reservas)
print("\n")
print("El precio mas bajo de las reservas es:")
print("\n")
reservas = bookings['total_amount'].min()
print("$", reservas)

El precio medio de las reservas es:


$ 79025.60581152869


El precio mas alto de las reservas es:


$ 1204500


El precio mas bajo de las reservas es:


$ 3400


##5. ¿Qué rangos de aeronaves hay? (ordenarlos descendentemente)

In [73]:
aircrafts_data = pd.read_sql("SELECT * FROM aircrafts_data", conn)

print("Los rangos son (Ordenados descendentemente):")
print("\n")
rangos_aviones = aircrafts_data['range'].unique()
rangos_aviones_descendente = sorted(rangos_aviones, reverse=True)
for rango in rangos_aviones_descendente:
    print(rango)

Los rangos son (Ordenados descendentemente):


11100
7900
6700
5700
5600
4200
3000
2700
1200
